# Imports

In [1]:
from collections import Counter

from tqdm import tqdm

from pfc_packages.gpf_slr import *
from pfc_packages.utils import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Hyperparameters

In [2]:
# --- Hyperparameters for Ground Plane Fitting (GPF) ---
NUM_LOWEST_POINTS = 10000  # Number of lowest elevation points used to estimate initial ground seed (LPR)
NUM_ITERATIONS = 5  # Number of iterations for plane refinement in GPF
SEED_HEIGHT_THRESHOLD = 0.4  # Max height above LPR to consider a point as a ground seed
PLANE_DISTANCE_THRESHOLD = 0.2  # Max distance from plane to classify a point as ground

# --- Parameters for Scan Line Run (SLR) clustering ---
SLR_RUN_DISTANCE_THRESHOLD = (
    0.2  # Max distance between consecutive points in a scanline to form a run
)
SLR_MERGE_THRESHOLD = 1.0  # Max distance between runs in adjacent scanlines to be considered the same cluster

# Run Example

In [3]:
dataset = Dataset("../../datasets/semantic-kitti", load_cluster=True)

clusters_features = []

for frame in tqdm(dataset, desc="Processando frames"):
    clusters_features.append(extract_features_from_clusters(frame, min_cluster_size=3))
    # break

clusters_features[0].shape

Processando frames: 100%|██████████| 5/5 [00:02<00:00,  1.76it/s]


(514, 10)

In [4]:
run_viz(
    clusters_features[0],
    cluster_viz=True,
    show_true_label=True,
    point_size=3,
    # show_min_max=True,
)